In [1]:
import pyranges as pr
import yaml


In [4]:
# config_file = s'../ad008/config.yml'
config_file = '../../../../config.yml'
with open(config_file) as f:
    config = yaml.safe_load(f)

In [12]:
ofile = 'Clu_talon_reformatted.gtf'
ifile = 'Clu_talon_observedOnly.gtf'

human_annot = '../../../../'+config['human_ref']['gtf']
annot = '../../../../'+config['ref']['gtf']

wc = {'mouse_gene': 'Clu'}

locus_type = 'human'

print(human_annot)
print(annot)

../../../../human_ref/annot.gtf
../../../../ref/annot.gtf


In [19]:
if 'mouse_gene' in list(wc.keys()):
        species = 'mouse'
        gene = wc['mouse_gene']
elif 'human_gene' in list(wc.keys()):
    species = 'human'
    gene = wc['human_gene']

new_gene = f'{species}_{gene}'

if gene == 'dummy':
    pathlib.Path(ofile).touch()
    # return

# for whole chrom, extract all GTF entries from the corresponding gene
elif locus_type == 'human':
    if species == 'human':
        gtf_file = human_annot
    elif species == 'mouse':
        gtf_file = annot
    df = pr.read_gtf(gtf_file).df
    df = df.loc[df.gene_name == gene]
    
# for other cases, use the mapped transcript TALON output
else:
    df = pr.read_gtf(ifile, rename_attr=True).df

    keep_cols = ['Chromosome', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand',
       'Frame', 'gene_id', 'gene_name', 'gene_status', 'source_attr',
       'transcript_id', 'transcript_status', 'transcript_name', 'exon_number', 'exon_id',
       'exon_status']
    df = df[keep_cols]

    df.loc[df.Feature.isin(['transcript','exon']), 'transcript_status'] = 'KNOWN'
    df.loc[df.Feature=='exon', 'exon_status'] = 'KNOWN'
    df.gene_status = 'KNOWN'
    
# assign these models new gene names and write to output
df.gene_id = new_gene
df.gene_name = new_gene

pr.PyRanges(df).to_gtf(ofile)

In [16]:
print(len(df.index))
df = df.loc[df.gene_name==gene]
print(len(df.index))

1853209
108


In [22]:
def refmt_mapped_transcript_gtf(wc, ifile, ofile):
    """
    Format a TALON gtf output from annotated transcripts mapped 
    onto pseudochromosomes in order to make them appear as known.
    
    # TODO - support whole chrom
    """
    if 'mouse_gene' in list(wc.keys()):
        species = 'mouse'
        gene = wc['mouse_gene']
    elif 'human_gene' in list(wc.keys()):
        species = 'human'
        gene = wc['human_gene']
    df = pr.read_gtf(ifile, rename_attr=True).df    
    
    keep_cols = ['Chromosome', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand',
       'Frame', 'gene_id', 'gene_name', 'gene_status', 'source_attr',
       'transcript_id', 'transcript_status', 'transcript_name', 'exon_number', 'exon_id',
       'exon_status']
    df = df[keep_cols]
    
    df.loc[df.Feature == 'transcript', 'transcript_status'] = 'KNOWN'
    df.gene_status = 'KNOWN'
    df.gene_id = gname
    df.gene_name = gname
    
    pr.PyRanges(df).to_gtf(ofile)

In [23]:
refmt_mapped_transcript_gtf(wc, ifile, ofile)

Found attributes with reserved names: ['source'].
Renaming attributes with suffix '_attr'


In [11]:
gname = 'hClu'
ofile = 'Clu_talon_reformatted.gtf'

df = pr.read_gtf('Clu_talon_observedOnly.gtf', rename_attr=True).df

Found attributes with reserved names: ['source'].
Renaming attributes with suffix '_attr'


In [12]:
# rm_cols = ['NIC_transcript', 'NNC_transcript', 'ISM_transcript',
#            'intergenic_transcript', 'antisense_transcript',
#             'ISM-prefix_to_IDs',
#              'ISM-prefix_transcript',
#              'ISM_to_IDs',
#              'ISM-suffix_to_IDs',
#              'ISM-suffix_transcript',
#            'antisense_gene',
#     'gene_antisense_to_IDs', 'intergenic_novel', 'fusion_transcript',
#            'fusion_novel', 'genomic_transcript']
# for c in rm_cols:
#     if c in df.columns.tolist():
#         df.drop(c, axis=1, inplace=True)
keep_cols = ['Chromosome', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand',
       'Frame', 'gene_id', 'gene_name', 'gene_status', 'source_attr',
       'transcript_id', 'transcript_status', 'transcript_name', 'exon_number', 'exon_id',
       'exon_status']
df = df[keep_cols]

In [13]:
df.gene_status.unique()

array(['NOVEL'], dtype=object)

In [14]:
df.transcript_status.unique()

array([nan, 'NOVEL'], dtype=object)

In [15]:
df.columns.tolist()

['Chromosome',
 'Source',
 'Feature',
 'Start',
 'End',
 'Score',
 'Strand',
 'Frame',
 'gene_id',
 'gene_name',
 'gene_status',
 'source_attr',
 'transcript_id',
 'transcript_status',
 'transcript_name',
 'exon_number',
 'exon_id',
 'exon_status']

In [16]:
df.loc[df.Feature == 'transcript', 'transcript_status'] = 'KNOWN'
df.gene_status = 'KNOWN'
df.gene_id = gname
df.gene_name = gname
df.head()

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,gene_status,source_attr,transcript_id,transcript_status,transcript_name,exon_number,exon_id,exon_status
0,hCLU,TALON,gene,3000,10510,.,+,.,hClu,hClu,KNOWN,TALON,NaN,NaN,NaN,NaN,NaN,NaN
1,hCLU,TALON,transcript,3000,15595,.,+,.,hClu,hClu,KNOWN,NaN,TALONT000141863,KNOWN,TALONT000141863,NaN,NaN,NaN
2,hCLU,TALON,exon,3000,3206,.,+,.,hClu,hClu,KNOWN,NaN,TALONT000141863,NOVEL,TALONT000141863,1,730959,NOVEL
3,hCLU,TALON,exon,6249,6370,.,+,.,hClu,hClu,KNOWN,NaN,TALONT000141863,NOVEL,TALONT000141863,2,730947,NOVEL
4,hCLU,TALON,exon,7842,7991,.,+,.,hClu,hClu,KNOWN,NaN,TALONT000141863,NOVEL,TALONT000141863,3,730949,NOVEL


In [17]:
pr.PyRanges(df).to_gtf(ofile)

In [9]:
# talon_initialize_database \
#     		--f Clu_talon_reformatted.gtf \
#     		--g test \
#     		--a test \
#     		--l 0 \
#     		--idprefix TALON \
#     		--5p 500 \
#     		--3p 300 \
#     		--o test

In [10]:
df.columns

Index(['Chromosome', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand',
       'Frame', 'gene_id', 'gene_name', 'gene_status', 'source_attr',
       'talon_gene', 'transcript_id', 'transcript_status', 'transcript_name',
       'talon_transcript', 'exon_number', 'exon_id', 'talon_exon',
       'exon_status'],
      dtype='object')

In [ ]:
keep_cols = ['Chromosome', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand',
       'Frame', 'gene_id', 'gene_name', 'gene_status', 'source_attr',
       'transcript_id', 'transcript_status', 'transcript_name', 'exon_number', 'exon_id',
       'exon_status']